In [1]:
import numpy as np
import random
import itertools
import math
import tqdm
from tqdm import trange
import time

# Helper Function

In [2]:
key_mapping={
    'C':0,
    'D':2,
    'E':4,
    'F':5,
    'G':7,
    'A':9,
    'B':11
}
def key2num(key):  
  key=key.upper()
  num=key_mapping[key[0]]
  modifier=len(key)
  if modifier==1:
    return num
  elif key[1]=='#':
    return (num+(modifier-1))%12
  elif key[1]=='B' or key[1]=='-':
    return (num-(modifier-1))%12
  elif key[1]=='X':
    return (num+(modifier-1)*2)%12

# key_list to number_list
def keys2num(keys):
    return [key2num(key) for key in keys]

In [3]:
# function to distingish whether notes in given timestamp and chord are within or outside the chord (0=outside,1=within)
import json
with open('../modules/json_files/keychorddict.json') as json_file:
    chord_notes = json.load(json_file)
  
    def note_2_class(chord,notes_at_t,chord_notes=chord_notes):
        notes_at_t=keys2num(notes_at_t)
        note_in_chord=chord_notes[chord]['idx']
        return [int(note in note_in_chord) for note in notes_at_t]

In [4]:
key_mapping = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}
changekey = {
    "GBMINOR": "F#MINOR",
    "DBMINOR": "C#MINOR",
    "ABMINOR": "G#MINOR",
    "A#MINOR": "BBMINOR",
    "D#MINOR": "EBMINOR",
    "A#MAJOR": "BBMAJOR",
    "D#MAJOR": "EBMAJOR",
    "G#MAJOR": "ABMAJOR",
    "C#MAJOR": "DBMAJOR",
    "F#MAJOR": "GBMAJOR",
}

# key_list to number_list
def keys2num(keys):
    
    # 1 by 1 key to number
    def key2num(key):
        key = key.upper()
        num = key_mapping[key[0]]
        modifier = len(key)
        if modifier == 1:
            return num
        elif key[1] == "#":
            return (num + (modifier - 1)) % 12
        elif key[1] == "B" or key[1] == "-":
            return (num - (modifier - 1)) % 12
        elif key[1] == "X":
            return (num + (modifier - 1) * 2) % 12

    return [key2num(key) for key in keys]

# HMM

In [5]:


class HMM:
    def __init__(self,no_of_state,no_of_value,states,values):
        #randomize all matrix, each row should sum to 1
        #self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_value) for i in range(no_of_state)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        

        self.zero=0.00000000000001
        self.emssion_matrix=np.random.rand(no_of_value)
        self.emssion_matrix=self.emssion_matrix/self.emssion_matrix.sum()
        if self.emssion_matrix[0]>self.emssion_matrix[1]:
            temp=self.emssion_matrix[0]
            self.emssion_matrix[0]=self.emssion_matrix[1]
            self.emssion_matrix[1]=temp
        self.emssion_matrix=np.array([self.emssion_matrix,]*len(states)) 
        self.emssion_matrix=np.array([[0.2,0.8],]*len(states)) 
        # assume same probability to emit "note within chord"  for all chords
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        self.no_of_state=no_of_state
        self.states=states
        self.values=values
        self.observered=None
        self.key=None
        
        self.probit_at_i_table=None
        self.probit_transit_i_j_table=None
        self.forward_table=None
        self.backward_table=None
        self.chord_probit=None
        self.note_probit=None
        
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emission_matrix\n',self.emssion_matrix)
        print("note prob\n",self.note_probit)
    
    def likelihood(self,state,ob_t,key):

        #key mask
        if key[-5:].lower()!=self.states[state][:5].lower():
            return np.log(self.zero)
        
        chord=key+self.states[state][5:]

        #prediction from note2chord
        a=noteToChord.NoteToChord(ob_t,key,20,0)

        #verify output number
        if key[-5:]=='Major':
            assert(len(a)==16)
        else:
            assert(len(a)==18)
       
        #normalize
        score = np.array([i['Score'] if 'Score' in i.keys() else 1 for i in a])
        score=[e if e>0 else self.zero for e in score]
        score/=sum(score)
        #score/=sum(score)
        #score=[(np.exp(-i*0.2)) for i in range(18)] #fixed template probit
        

        #get the idx of interested chord(state)
        chords=[i['Chord'] for i in a]
        idx=chords.index(chord)
        
        #probit to emit chordNote/non-chordNote
        observation=ob_t.keys()
        obs_weight=[ob_t[e] for e in observation]
      #  obs_2=note_2_class(chord,observation) #2 class
      #  prob=0
        note_prob=0
        obs_no=keys2num(observation)
        for i,x in enumerate(obs_no):
            note_prob+=self.note_probit[x]*obs_weight[i]
     #   for i,x in enumerate(obs_2):
      #      prob+=(self.emssion_matrix[state][x])*obs_weight[i]

        assert(score[idx]>0)
        #assert(prob>0)
            #   N2C P(chord|note)*  P(note)        / P(chord)
        return np.log(score[idx])+np.log(note_prob)-np.log(self.chord_probit[state]) # +np.log(prob)

    def forward(self,t,j,ob=None,mode=False):
        if ob is None:
            ob=self.observered
        if np.isnan(self.forward_table[t][j]).any():
            if t==0:
                if mode==True:
                    self.forward_table[t][j]=[(np.log(self.initial_matrix[j]))+self.likelihood(j,ob[t],self.key[t]),0]
                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
            else:          
                if mode==True:                
                    result=np.array([self.forward(t-1,i,ob,mode)[0]+np.log(self.transition_matrix[i][j]) for i in range(self.no_of_state)])+self.likelihood(j,ob[t],self.key[t])
                    self.forward_table[t][j]=[np.max(result),np.argmax(result)]
                    return self.forward_table[t][j][0],self.forward_table[t][j][1]
        else:
            return self.forward_table[t][j]
    
    def backward(self,t,i,ob=None):
        if ob is None:
            ob=self.observered
        if np.isnan(self.backward_table[t][i]):
            if t==len(ob)-1:
                self.backward_table[t][i]=1
                return self.backward_table[t][i]
            else:
                self.backward_table[t][i]=sum([self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob) for j in range(self.no_of_state)])
                return self.backward_table[t][i]
        else:
            return self.backward_table[t][i]
            
           
    def probit_at_i(self,t,i,ob=None):#Gamma γt(i) = P(qt = i|O,λ)      
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_at_i_table[t][i]):
            numerator=self.forward(t,i,ob)*self.backward(t,i,ob)#sum probability of all path passing through state[i] at time t
            denominator=sum([self.forward(t,j,ob)*self.backward(t,j,ob) for j in range(self.no_of_state)]) #prob of passing through  ALL_state at time t
            self.probit_at_i_table[t][i]=numerator/denominator
            return self.probit_at_i_table[t][i]
        else:
            return self.probit_at_i_table[t][i]
    
    def probit_transit_i_j(self,t,i,j,ob=None):#epsilon ξt(i, j) = P(qt = i,qt+1 = j|O,λ)
        if ob is None:
            ob=self.observered
        if np.isnan(self.probit_transit_i_j_table[t][i][j]):
            numerator=self.forward(t,i,ob)*self.transition_matrix[i][j]*self.likelihood(j,ob[t+1])*self.backward(t+1,j,ob)#sum probability of all path transit from state[i] to state[j] at time t
            denominator=sum([sum([self.forward(t,m,ob)*self.transition_matrix[m][n]*self.likelihood(n,ob[t+1])*self.backward(t+1,n,ob) for n in range(self.no_of_state)]) for m in range(self.no_of_state)]) #prob of ALL transition combination at time t
            self.probit_transit_i_j_table[t][i][j]=(numerator/denominator)
            return self.probit_transit_i_j_table[t][i][j]
        else:
            return self.probit_transit_i_j_table[t][i][j]
    
    #modify from https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm/9730083 , author RBF06(https://stackoverflow.com/users/3311728/rbf06)
    def predict(self,ob,test_key):  
        self.forward_table=np.empty((len(ob),self.no_of_state,2))
        self.forward_table[:]= np.NaN
        self.key=test_key
        self.chord_probit=self.transition_matrix.sum(axis=0)
        self.chord_probit/=self.chord_probit.sum()
        print(self.chord_probit)
 

        T1=np.empty((self.no_of_state,len(ob)),'d')
        T2=np.empty((self.no_of_state,len(ob)),'B')
        for idx in range(self.no_of_state):
            T1[idx,0]=self.forward(0,idx,ob,True)[0]
        T2[:,0]=0
        
        for i in range(1,len(ob)):
            for idx in range(self.no_of_state):
                T1[idx,i],T2[idx,i]=self.forward(i,idx,ob,True)

        x = np.empty(len(ob), 'B')
        x[-1] = np.argmax(T1[:, len(ob) - 1])
        
        #backtracking
        for i in reversed(range(1, len(ob))):
            x[i - 1] = T2[x[i], i]
        return x
    
    def get_detail(self,label):
        key='Major' if 'M' in label else 'Minor'
        chord=label.split("_")[1]
        key_note=label.split('M')[0] if key=='Major' else label.split('m')[0].lower()
        if key=='Major':
            if key_note=='F#':
                key_note='Gb'
            elif key_note=='A#':
                key_note='Bb'
            elif key_note=='C#':
                key_note='Db'
            elif key_note=='G#':
                key_note='Ab'
            elif key_note=='D#':
                key_note='Eb'
        else:
            if key_note=='gb':
                key_note='f#'
            elif key_note=='db':
                key_note='c#'
            elif key_note=='ab':
                key_note='g#'
            elif key_note=='d#':
                key_note='eb'
                
        if 'German' in chord:
            chord=key+'GerVI'
        elif 'Dim7' in chord:
            chord=key+'DimVII7'
        elif 'FrenchVI' in chord:
            chord=key+'FreVI'
        else:
            chord=key+chord
        return chord,key_note
    
    def train_supervisied(self,obs,labels): # by MLE
        
        initial_matrix= np.zeros(self.no_of_state)
        emssion_matrix=np.zeros((2)) 
        transition_matrix=np.array([np.zeros(self.no_of_state) for i in range(self.no_of_state)])
        note_list=np.zeros(12)

        for idx_1,label in enumerate(labels):#loop each score
            for idx_2,lab in enumerate(label):

                chord,key_name=self.get_detail(label[idx_2])
                if idx_2==0:
                    initial_matrix[self.states.index(chord)]+=1
                else:
                    Pre_chord,Pre_key_name=self.get_detail(label[idx_2-1])
                    if '+' in Pre_chord and 'Major' in Pre_chord:
                        Pre_chord=Pre_chord.replace('+','')
                    if '+' in chord and 'Major' in chord:
                        chord=chord.replace('+','')
                    transition_matrix[self.states.index(Pre_chord)][self.states.index(chord)]+=1
                    ob_t_no=keys2num(obs[idx_1][idx_2])
                    ob_t=note_2_class(key_name+chord,obs[idx_1][idx_2]) #2 class
                    for x in ob_t:
                        emssion_matrix[x]+=1
                    for x in ob_t_no:
                        note_list[x]+=1
                        
        #save back to model
        self.initial_matrix=np.array([item/initial_matrix.sum() if item >0 else self.zero for item in initial_matrix])
        self.transition_matrix=np.array([row/row.sum() if row.sum()>0 else row+self.zero for row in transition_matrix])
        for row in range(self.transition_matrix.shape[0]):
            for col in range(self.transition_matrix.shape[1]):
                if self.transition_matrix[row][col]==0:
                    self.transition_matrix[row][col]=self.zero
        self.emssion_matrix=np.array([emssion_matrix/emssion_matrix.sum()] *self.no_of_state) 
        self.note_probit=note_list/sum(note_list)

      
    def train(self,obs,key_name,epochs=2):
        #O:observed values
        #λ:model parameters

        
        for epoch in range(epochs):
            for ob_idx,ob in enumerate(obs):

                print('running',ob_idx,key_name[ob_idx].lower()[:-1])
                
                self.probit_at_i_table=np.empty((len(ob),self.no_of_state))
                self.probit_transit_i_j_table=np.empty((len(ob),self.no_of_state,self.no_of_state))
                self.forward_table=np.empty((len(ob),self.no_of_state))
                self.backward_table=np.empty((len(ob),self.no_of_state))
                self.probit_at_i_table[:]= np.NaN
                self.probit_transit_i_j_table[:]=np.NaN
                self.forward_table[:]= np.NaN
                self.backward_table[:]=np.NaN

                #initalize DP table
                for t, i in tqdm.tqdm(itertools.product(range(len(ob)),range(self.no_of_state))):
                    self.forward(t,i,ob)
                    self.backward(t,i,ob)
                    self.probit_at_i(t,i,ob)
                    for j in range(self.no_of_state):
                        if t!=len(ob)-1:
                            self.probit_transit_i_j(t,i,j,ob)
                
                
                
                #initial matrix
                #for i in range(self.no_of_state):
                #    self.initial_matrix[i]=self.probit_at_i(0,i,ob)
                #    if self.initial_matrix[i]==0:
                #        self.initial_matrix[i]=0.00000001
                
                #transition matrix
                for i, j in itertools.product(range(self.no_of_state),range(self.no_of_state)):
                    self.transition_matrix[i][j]=sum([self.probit_transit_i_j(t,i,j,ob) for t in range(len(ob)-1)])/sum([self.probit_at_i(t,i,ob) for t in range(len(ob)-1)])
                    if self.transition_matrix[i][j]==0 :
                        self.transition_matrix[i][j]=self.zero
                #emission matrix
                #for j, k in itertools.product(range(self.no_of_state),range(len(self.values))):   
                #    total=0
                #    
                #    #Modification: convert notes to 2 classes (outside or inside given chord)
                #    chord=self.states[j]  #numeric to chord name
                #    ob_2_class=[note_2_class(chord,ob_t) for ob_t in ob] #2 class

                #    for t in range(len(ob)):
                #        if k in ob_2_class[t]:
                #            #Modification: multiple by how many times do k appear at timestamp t
                #            total+=self.probit_at_i(t,j,ob)*ob_2_class[t].count(k)  
                            
                #    #Modification: multiple by len(ob[t]), which is the total length of notes at timestamp t                    
                #    self.emssion_matrix[:,k]=total/sum([self.probit_at_i(t,j,ob)*len(ob[t]) for t in range(len(ob))])  
                    
                #    #smoothing
                    
                #    if self.emssion_matrix[j][k]==0 :
                #        self.emssion_matrix[:,k]=0.00000001
                     

# Prepare Data

In [6]:
import json
with open('../data/training_data2.json') as json_file:
    data = json.load(json_file)

In [7]:
#shuffle
score=np.array(list(data.keys()))
#np.random.shuffle(score)
test_score=score[:5]
score=score[5:]
test_score,score

(array(['Etude_in_C_Minor.mxl', 'Etude_in_F_Major.mxl',
        'Etude_in_F_Minor.mxl', 'Etude_in_Gb_Major.mxl',
        'Etude_in_Gb_Major_Opus_25.mxl'], dtype='<U51'),
 array(['Il_Vecchio_Castello.mxl', 'Menuet_in_G_Minor.mxl',
        'Minuet_in_F.mxl', 'Minuet_in_G_Major_2nd.mxl',
        'Moonlight_Sonata_1st_Movement.mxl', 'Nocturne_in_B_Major.mxl',
        'Nocturne_in_C_Minor.mxl', 'Nocturne_in_Eb_Major.mxl',
        'Nocturne_in_E_Major.mxl', 'Nocturne_in_E_Minor.mxl',
        'Nocturne_in_F_Major.mxl', 'Nocturne_in_F_Minor.mxl',
        'Nocturne_No._20_in_C_Minor.mxl', 'notestochord.mxl',
        'Piano_Sonata_No._11.mxl', 'Prelude_in_A_Major.mxl',
        'Prelude_in_B_Major.mxl', 'Prelude_in_B_Minor.mxl',
        'Prelude_in_B_Minor_Opus104a.mxl', 'Prelude_in_C_Minor.mxl',
        'Prelude_in_Db_Major.mxl', 'Prelude_in_F_Major.mxl',
        'Prelude_in_G_Major.mxl', 'Prlude_Opus_28_No._4_in_E_Minor.mxl',
        'Sonata_No._1.mxl', 'Sonate_No._28.mxl',
        'Sonate_No._

In [8]:
#prepare train data
score_name=score
x_train=[]
y_train=[]
for idx_root,name in enumerate(score_name):
    
    training=[]
    training_label=[]

    training=data[name]['note_seq']
    training_label=data[name]['chord_seq']
    
    x_train.append(training)
    y_train.append(training_label)

In [9]:
#prepare test data (1 score)
test_score=test_score
score_test_choice=0

test_sample=[]
testing_label=[]
name=test_score[score_test_choice]

test_sample=data[name]['note_seq']
testing_label=data[name]['chord_seq']

test_key=[]
for e in testing_label:
    temp_key=e.split('_')[0]
    temp_name=temp_key[:-1]
    temp_key=temp_key[-1]
    if temp_key=='m':
        test_key.append(temp_name.lower()+'Minor')
    else:
        test_key.append(temp_name[0].upper()+temp_name[1:].lower()+'Major')   
for idx,e in enumerate(test_key):
    if 'F#Major' in e:
        test_key[idx]=test_key[idx].replace("F#Major", "GbMajor")

# Training

In [10]:
h_states=['MinorI', 'MinorI+',
          'MinorbII', 'MinorII', 'MinorII7',
          'MinorIII',
          'MinorIV', 'MinorIV+',
          'MinorV', 'MinorV+', 'MinorV+7',
          'MinorVI', 'MinorGerVI', 'MinorFreVI', 'MinorItaVI',
          'MinorVII', 'MinorDimVII7',#, 'MinorDimVII'
          'MajorI',
          'MajorbII','MajorII','MajorII7',
          'MajorIII',
          'MajorIV',
          'MajorV','MajorV7',
          'MajorbVI','MajorGerVI','MajorFreVI','MajorItaVI','MajorVI',
          'MajorVII','MajorVII7','MajorDimVII7'
]
#all possible chordm will add the key to the chord afterward e.g. cMinorI

In [11]:
test=HMM(len(h_states),2,h_states,["outside chord","inside chord"])

In [12]:
test.train_supervisied(x_train,y_train)#initialize parameter

In [13]:
test.debug()

initial_matrix
 [4.24242424e-01 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 3.33333333e-01 1.00000000e-14 3.03030303e-02
 1.00000000e-14 1.00000000e-14 1.00000000e-14 2.12121212e-01
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14 1.00000000e-14 1.00000000e-14 1.00000000e-14
 1.00000000e-14]
transition_matrix
 [[1.98412698e-02 2.38095238e-02 2.18253968e-02 ... 1.98412698e-03
  1.00000000e-14 5.95238095e-03]
 [1.66666667e-01 4.16666667e-02 4.16666667e-02 ... 1.00000000e-14
  1.00000000e-14 1.00000000e-14]
 [2.77777778e-01 2.77777778e-02 1.00000000e-14 ... 1.00000000e-14
  1.00000000e-14 1.00000000e-14]
 ...
 [1.00000000e-14 1.00000000e-14 1.00000000e-14 ... 1.00000000e-14
  1.00000000e-14 1.00000000e-14]
 [1.00000000e-14 1.00000000e-14 1.00000000e-14 ... 1.000000

In [14]:
import sys
sys.path.append('../modules')
import noteToChordWeighted  as noteToChord

In [15]:
chord_notes['CMajorVII7']

{'idx': [11, 2, 5, 9],
 'naming': ['B', 'D', 'F', 'A'],
 'chord': 'VII7',
 'key': 'CMajor'}

# Evaluation

In [16]:
len(test_key)

85

In [17]:
len(test_sample)

85

In [18]:
def get_test(ii=0):
    #prepare test data (1 score)
    score_test_choice=ii


    name=test_score[score_test_choice].copy()

    test_sample=data[name]['note_seq'].copy()
    testing_label=data[name]['chord_seq'].copy()
    test_key=[]
    for e in testing_label:
        temp_key=e.split('_')[0]
        temp_name=temp_key[:-1]

        temp_key=temp_key[-1]
        if temp_key=='m':
            test_key.append(temp_name.lower()+'Minor')
        else:

            test_key.append(temp_name[0].upper()+temp_name[1:].lower()+'Major')   
    for idx,e in enumerate(test_key):
        if 'F#Major' in e:
            test_key[idx]=test_key[idx].replace("F#Major", "GbMajor")
        if 'G#Major' in e:
            test_key[idx]=test_key[idx].replace("G#Major", "AbMajor")
        if 'dbMinor' in e:
            test_key[idx]=test_key[idx].replace("dbMinor", "c#Minor")
        if 'd#Minor' in e:
            test_key[idx]=test_key[idx].replace("d#Minor", "ebMinor")
        if 'C#Major' in e:
            test_key[idx]=test_key[idx].replace("C#Major", "DbMajor")
        if 'A#Major' in e:
            test_key[idx]=test_key[idx].replace("A#Major", "BbMajor")
        if 'abMinor' in e:
            test_key[idx]=test_key[idx].replace("abMinor", "g#Minor")
        if 'gbMinor' in e:
            test_key[idx]=test_key[idx].replace("gbMinor", "f#Minor")
    remove_idx=[]
    
    test_sample=np.array(test_sample)
    test_key=np.array(test_key)
    for idx,e in enumerate(test_sample):
        if len(e)<=1:
            remove_idx.append(idx)
    print(remove_idx)
    if len(remove_idx)>0:
        test_sample=np.delete(test_sample,remove_idx)
        test_key=np.delete(test_key,remove_idx)

    prediction=test.predict(test_sample,test_key)
    prediction=[h_states[i] for i in prediction]
    for idx,e in enumerate(testing_label):
        if 'M' in e:
            testing_label[idx]=testing_label[idx].replace("M_", "Major")
            testing_label[idx]=testing_label[idx][0].upper()+testing_label[idx][1:]
        else:
            testing_label[idx]=testing_label[idx].replace("m_", "Minor")
            testing_label[idx]=testing_label[idx][0].lower()+testing_label[idx][1:]
    #compare with chord2note
    note2chord_ans=[]
    for notes,key in zip(test_sample,test_key):
        note2chord_ans.append(noteToChord.NoteToChord(notes,key,1,0)[0]['Chord'])

    for idx,e in enumerate(testing_label):
        testing_label[idx]='M'+e.split('M')[1]
    for idx,e in enumerate(note2chord_ans):
        note2chord_ans[idx]='M'+e.split('M')[1]
    #evaluation
    correct=0
    wrong=0
    for idx,label in enumerate(note2chord_ans):

        if label.replace('7','').replace('Gb','F#').upper()==testing_label[idx].replace('7','').replace('German','Ger').replace('Dim','DimVII').upper():
            correct+=1
        else:
            wrong+=1
    print('N2C acc',correct/(correct+wrong),correct,wrong)   
    #compare with 2-HMM
    #evaluation
    correct=0
    wrong=0
    for idx,label in enumerate(prediction):
        if label.replace('7','').upper()==testing_label[idx].replace('7','').replace('German','Ger').replace('Dim','DimVII').upper():
            correct+=1
        else:
            wrong+=1

    print('HMM acc',correct/(correct+wrong),correct,wrong)

In [27]:
for i in range(len(test_score)):
    print('---------------',test_score[i],'---------------------------')
    get_test(i)
    print('---------------------------------------')

--------------- Etude_in_C_Minor.mxl ---------------------------
[]
[1.57243511e-01 1.77558921e-02 1.36970470e-02 5.97345903e-02
 1.22222222e-14 3.98394792e-02 4.15105582e-02 5.18209500e-03
 1.60180810e-02 8.43550675e-02 5.65299523e-02 1.88148993e-02
 8.36713997e-03 1.42450142e-03 1.22222222e-14 1.11333037e-02
 1.59979627e-02 2.23973724e-01 7.83996501e-04 1.11001383e-02
 1.22222222e-14 1.34140976e-03 2.26179001e-02 1.55570503e-01
 1.15560178e-04 1.21432908e-04 3.76462099e-03 1.22222222e-14
 1.22222222e-14 1.96544183e-02 9.24340480e-04 1.22222222e-14
 1.24278751e-02]
N2C acc 0.8588235294117647 73 12
HMM acc 0.2235294117647059 19 66
---------------------------------------
--------------- Etude_in_F_Major.mxl ---------------------------
[]
[1.57243511e-01 1.77558921e-02 1.36970470e-02 5.97345903e-02
 1.22222222e-14 3.98394792e-02 4.15105582e-02 5.18209500e-03
 1.60180810e-02 8.43550675e-02 5.65299523e-02 1.88148993e-02
 8.36713997e-03 1.42450142e-03 1.22222222e-14 1.11333037e-02
 1.599796

# check specific score

In [28]:
score_test_choice=0


name=test_score[score_test_choice].copy()

test_sample=data[name]['note_seq'].copy()
testing_label=data[name]['chord_seq'].copy()
test_key=[]
for e in testing_label:
    temp_key=e.split('_')[0]
    temp_name=temp_key[:-1]

    temp_key=temp_key[-1]
    if temp_key=='m':
        test_key.append(temp_name.lower()+'Minor')
    else:

        test_key.append(temp_name[0].upper()+temp_name[1:].lower()+'Major')   
for idx,e in enumerate(test_key):
    if 'F#Major' in e:
        test_key[idx]=test_key[idx].replace("F#Major", "GbMajor")
    if 'G#Major' in e:
        test_key[idx]=test_key[idx].replace("G#Major", "AbMajor")
    if 'dbMinor' in e:
        test_key[idx]=test_key[idx].replace("dbMinor", "c#Minor")
    if 'd#Minor' in e:
        test_key[idx]=test_key[idx].replace("d#Minor", "ebMinor")
    if 'C#Major' in e:
        test_key[idx]=test_key[idx].replace("C#Major", "DbMajor")
    if 'A#Major' in e:
        test_key[idx]=test_key[idx].replace("A#Major", "BbMajor")
    if 'abMinor' in e:
        test_key[idx]=test_key[idx].replace("abMinor", "g#Minor")
    if 'gbMinor' in e:
        test_key[idx]=test_key[idx].replace("gbMinor", "f#Minor")
remove_idx=[]

test_sample=np.array(test_sample)
test_key=np.array(test_key)
for idx,e in enumerate(test_sample):
    if len(e)<=1:
        remove_idx.append(idx)
print(remove_idx)
if len(remove_idx)>0:
    test_sample=np.delete(test_sample,remove_idx)
    test_key=np.delete(test_key,remove_idx)


[]


In [29]:
for idx,e in enumerate(test.states):
    print(e,test.chord_probit[idx])

MinorI 0.157243510688092
MinorI+ 0.017755892069336848
MinorbII 0.013697047004692743
MinorII 0.0597345903105804
MinorII7 1.2222222222218444e-14
MinorIII 0.039839479235950576
MinorIV 0.04151055817443108
MinorIV+ 0.005182094996918203
MinorV 0.01601808101821503
MinorV+ 0.08435506749890154
MinorV+7 0.056529952296142404
MinorVI 0.018814899308948797
MinorGerVI 0.008367139972308684
MinorFreVI 0.001424501424512835
MinorItaVI 1.2222222222218444e-14
MinorVII 0.0111333036721289
MinorDimVII7 0.015997962662353215
MajorI 0.22397372390834586
MajorbII 0.0007839965014834982
MajorII 0.011100138322543923
MajorII7 1.2222222222218444e-14
MajorIII 0.0013414097584046744
MajorIV 0.022617900083475785
MajorV 0.15557050319634896
MajorV7 0.00011556017797449004
MajorbVI 0.00012143290832996844
MajorGerVI 0.0037646209911940003
MajorFreVI 1.2222222222218444e-14
MajorItaVI 1.2222222222218444e-14
MajorVI 0.019654418264976926
MajorVII 0.0009243404802174826
MajorVII7 1.2222222222218444e-14
MajorDimVII7 0.01242787507311787

In [30]:
prediction=test.predict(test_sample,test_key)
prediction=[h_states[i] for i in prediction]

[1.57243511e-01 1.77558921e-02 1.36970470e-02 5.97345903e-02
 1.22222222e-14 3.98394792e-02 4.15105582e-02 5.18209500e-03
 1.60180810e-02 8.43550675e-02 5.65299523e-02 1.88148993e-02
 8.36713997e-03 1.42450142e-03 1.22222222e-14 1.11333037e-02
 1.59979627e-02 2.23973724e-01 7.83996501e-04 1.11001383e-02
 1.22222222e-14 1.34140976e-03 2.26179001e-02 1.55570503e-01
 1.15560178e-04 1.21432908e-04 3.76462099e-03 1.22222222e-14
 1.22222222e-14 1.96544183e-02 9.24340480e-04 1.22222222e-14
 1.24278751e-02]


In [31]:
#check prediction and label side by side
for i in range(len(prediction)):
    print(prediction[i],testing_label[i])

MinorI Cm_I
MinorIV Cm_II
MinorVII Cm_IV
MinorIII Cm_I
MinorVII Cm_II
MinorIII Cm_Dim7
MinorGerVI Cm_I
MinorFreVI Cm_V+
MinorII Fm_Dim7
MinorI Fm_I+
MinorDimVII7 Cm_II
MinorGerVI Cm_I
MinorFreVI Cm_II
MinorII Cm_IV
MinorI+ Cm_I+
MinorIV Cm_bII
MinorVII Cm_V+
MinorIII Cm_I+
MinorVII Cm_V+
MinorIII Cm_I+
MinorVII Cm_V+
MinorIII Cm_V
MinorbII Cm_IV+
MinorVI Cm_IV
MinorIV+ Cm_I+
MajorI AbM_I
MajorV7 AbM_V
MajorI AbM_I
MajorV7 AbM_V
MajorI AbM_IV
MajorIII AbM_I
MajorDimVII7 GM_V
MajorGerVI GM_I
MinorGerVI Cm_VI
MinorDimVII7 Cm_II
MinorDimVII7 Cm_V+
MinorDimVII7 Cm_IV
MinorDimVII7 Gm_Dim7
MinorGerVI Gm_I
MinorDimVII7 Cm_Dim7
MinorGerVI Cm_IV
MinorFreVI Cm_VI
MinorII Cm_II
MinorDimVII7 Cm_V+7
MinorGerVI Cm_VI
MinorDimVII7 Dm_V+
MinorDimVII7 Dm_Dim7
MinorDimVII7 Cm_V+7
MinorGerVI Cm_I
MinorFreVI Cm_II
MinorII Cm_IV
MinorI Cm_I
MinorDimVII7 Cm_II
MinorGerVI Cm_I
MinorFreVI Cm_V+
MinorII Fm_Dim7
MinorI Fm_I+
MinorGerVI Cm_Dim7
MinorFreVI Cm_I
MinorII Cm_II
MinorI+ Cm_I
MinorIV Cm_bII
MinorDimVII

In [32]:
for idx,e in enumerate(testing_label):
    if 'M' in e:
        testing_label[idx]=testing_label[idx].replace("M_", "Major")
        testing_label[idx]=testing_label[idx][0].upper()+testing_label[idx][1:]
    else:
        testing_label[idx]=testing_label[idx].replace("m_", "Minor")
        testing_label[idx]=testing_label[idx][0].lower()+testing_label[idx][1:]

In [42]:
#compare with chord2note
note2chord_ans=[]
for notes,key in zip(test_sample,test_key):
    note2chord_ans.append([e['Chord'] for e in noteToChord.NoteToChord(notes,key,4,0)])
    
#for idx,e in enumerate(testing_label):
#    testing_label[idx]='M'+e.split('M')[1]
#for idx,e in enumerate(note2chord_ans):
#    note2chord_ans[idx]='M'+e.split('M')[1]
#evaluation
correct=0
wrong=0

for idx,label in enumerate(note2chord_ans):
    
   # if label.replace('7','').replace('Gb','F#').upper()==testing_label[idx].replace('7','').replace('German','Ger').replace('Dim','DimVII').upper():
 #       correct+=1
 #   else:
 #       wrong+=1
        #print(label,testing_label[idx])
    print(label,prediction[idx],testing_label[idx])
print('acc',correct/(correct+wrong),correct,wrong)   

['cMinorI', 'cMinorI+', 'cMinorIII', 'cMinorVI'] MinorI MinorI
['cMinorII7', 'cMinorFreVI', 'cMinorII', 'cMinorIV'] MinorIV MinorII
['cMinorIV', 'cMinorII7', 'cMinorIV+', 'cMinorVI'] MinorVII MinorIV
['cMinorI', 'cMinorI+', 'cMinorIII', 'cMinorVI'] MinorIII MinorI
['cMinorII7', 'cMinorFreVI', 'cMinorII', 'cMinorIV'] MinorVII MinorII
['cMinorVI', 'cMinorGerVI', 'cMinorI', 'cMinorIII'] MinorIII MinorDim7
['cMinorI', 'cMinorI+', 'cMinorIII', 'cMinorVI'] MinorGerVI MinorI
['cMinorV+', 'cMinorV+7', 'cMinorDimVII7', 'cMinorV'] MinorFreVI MinorV+
['fMinorII', 'fMinorII7', 'fMinorDimVII7', 'fMinorV+7'] MinorII MinorDim7
['fMinorI+', 'fMinorI', 'fMinorV', 'fMinorV+'] MinorI MinorI+
['cMinorDimVII7', 'cMinorIV', 'cMinorII7', 'cMinorDimVII'] MinorDimVII7 MinorII
['cMinorI', 'cMinorI+', 'cMinorIII', 'cMinorVI'] MinorGerVI MinorI
['cMinorII7', 'cMinorFreVI', 'cMinorII', 'cMinorIV'] MinorFreVI MinorII
['cMinorIV', 'cMinorII7', 'cMinorIV+', 'cMinorVI'] MinorII MinorIV
['cMinorI+', 'cMinorI', 'cMinorV

ZeroDivisionError: division by zero

In [34]:
#compare with 2-HMM
#evaluation
correct=0
wrong=0
for idx,label in enumerate(prediction):
    if label.replace('7','').upper()==testing_label[idx].replace('7','').replace('German','Ger').replace('Dim','DimVII').upper():
        correct+=1
    else:
        wrong+=1
        print(label,testing_label[idx])
    
print('acc',correct/(correct+wrong),correct,wrong)

MinorIV MinorII
MinorVII MinorIV
MinorIII MinorI
MinorVII MinorII
MinorIII MinorDim7
MinorGerVI MinorI
MinorFreVI MinorV+
MinorII MinorDim7
MinorI MinorI+
MinorDimVII7 MinorII
MinorGerVI MinorI
MinorFreVI MinorII
MinorII MinorIV
MinorIV MinorbII
MinorVII MinorV+
MinorIII MinorI+
MinorVII MinorV+
MinorIII MinorI+
MinorVII MinorV+
MinorIII MinorV
MinorbII MinorIV+
MinorVI MinorIV
MinorIV+ MinorI+
MajorI MajorIV
MajorIII MajorI
MajorDimVII7 MajorV
MajorGerVI MajorI
MinorGerVI MinorVI
MinorDimVII7 MinorII
MinorDimVII7 MinorV+
MinorDimVII7 MinorIV
MinorGerVI MinorI
MinorGerVI MinorIV
MinorFreVI MinorVI
MinorDimVII7 MinorV+7
MinorGerVI MinorVI
MinorDimVII7 MinorV+
MinorDimVII7 MinorV+7
MinorGerVI MinorI
MinorFreVI MinorII
MinorII MinorIV
MinorDimVII7 MinorII
MinorGerVI MinorI
MinorFreVI MinorV+
MinorII MinorDim7
MinorI MinorI+
MinorGerVI MinorDim7
MinorFreVI MinorI
MinorI+ MinorI
MinorIV MinorbII
MinorFreVI MinorV+7
MinorVII MinorV+
MinorIII MinorI
MinorVII MinorII
MinorIII MinorI
MinorVII M